# Emotion recognition with a transformer-based encoder
Input is text of conversations

In [13]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

notebooks_dir = Path.cwd()
project_dir = notebooks_dir.parent
data_dir = project_dir / 'data' / 'raw'
text_data_path = data_dir / 'Subtask_2_train.json'

In [2]:
text_data = json.loads(text_data_path.read_text())
for i in range(len(text_data[0]['conversation'])):
    print(text_data[0]['conversation'][i])

{'utterance_ID': 1, 'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .', 'speaker': 'Chandler', 'emotion': 'neutral', 'video_name': 'dia1utt1.mp4'}
{'utterance_ID': 2, 'text': 'Oh , yeah . Had that dream .', 'speaker': 'All', 'emotion': 'neutral', 'video_name': 'dia1utt2.mp4'}
{'utterance_ID': 3, 'text': 'Then I look down , and I realize there is a phone ... there .', 'speaker': 'Chandler', 'emotion': 'surprise', 'video_name': 'dia1utt3.mp4'}
{'utterance_ID': 4, 'text': 'Instead of ... ?', 'speaker': 'Joey', 'emotion': 'surprise', 'video_name': 'dia1utt4.mp4'}
{'utterance_ID': 5, 'text': 'That is right .', 'speaker': 'Chandler', 'emotion': 'anger', 'video_name': 'dia1utt5.mp4'}
{'utterance_ID': 6, 'text': 'Never had that dream .', 'speaker': 'Joey', 'emotion': 'neutral', 'video_name': 'dia1utt6.mp4'}
{'utterance_ID': 7, 'text': 'No .', 'speaker': 'Phoebe', 'emotion': 'neutral', 'video_name': 'dia1utt7.mp4'}

In [6]:
def print_format_conv(json_convo):
    for i in range(len(json_convo)):
        print(f"{json_convo[i]['speaker']} ({json_convo[i]['emotion']}): {json_convo[i]['text']}")

print_format_conv(text_data[0]['conversation'])

Chandler (neutral): Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .
All (neutral): Oh , yeah . Had that dream .
Chandler (surprise): Then I look down , and I realize there is a phone ... there .
Joey (surprise): Instead of ... ?
Chandler (anger): That is right .
Joey (neutral): Never had that dream .
Phoebe (neutral): No .
Chandler (neutral): All of a sudden , the phone starts to ring .


In [12]:
# extract conversation lines and put them into a single string with special sol and eol tokens
# and extract emotions to list 
def get_conversation_from_json(json_convo):
    convo = ''
    emotions = []
    for line_idx in range(len(json_convo)):
        convo += ' <sol> ' + json_convo[line_idx]['text'] + ' <eol> '
        emotions.append(json_convo[line_idx]['emotion'])
    return convo, emotions

convos = []
all_emotions = []
for convo_idx in range(len(text_data)):
    convo, emotions = get_conversation_from_json(text_data[convo_idx]['conversation'])
    convos.append(convo)
    all_emotions.append(emotions)

data = {'conversation' : convos, 'emotions' : all_emotions}
print(data)

{'conversation': [' <sol> Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked . <eol>  <sol> Oh , yeah . Had that dream . <eol>  <sol> Then I look down , and I realize there is a phone ... there . <eol>  <sol> Instead of ... ? <eol>  <sol> That is right . <eol>  <sol> Never had that dream . <eol>  <sol> No . <eol>  <sol> All of a sudden , the phone starts to ring . <eol> ', ' <sol> I do not want to be single , okay ? I just ... I just ... I just wanna be married again ! <eol>  <sol> And I just want a million dollars ! <eol>  <sol> Rachel ? ! <eol> ', ' <sol> Oh my God ! <eol>  <sol> I know , I know , I am such an idiot . <eol>  <sol> I guess I should have caught on when she started going to the dentist four and five times a week . I mean , how clean can teeth get ? <eol>  <sol> My brother going through that right now , he is such a mess . How did you get through it ? <eol>  <sol> Well , you might try accidentally break

In [15]:
df = pd.DataFrame.from_dict(data)
df.head()

,conversation,emotions
0,"<sol> Alright , so I am back in high school ,...","[neutral, neutral, surprise, surprise, anger, ..."
1,"<sol> I do not want to be single , okay ? I j...","[sadness, neutral, surprise]"
2,"<sol> Oh my God ! <eol> <sol> I know , I kno...","[surprise, neutral, sadness, neutral, neutral,..."
3,"<sol> Barry , I am sorry ... <eol> <sol> I a...","[sadness, sadness, sadness]"
4,"<sol> Oh , look , wish me luck ! <eol> <sol>...","[joy, neutral, joy]"


In [16]:
# imports for text cleaning
import contractions
import re
import nltk
from nltk.corpus import stopwords 
from nltk.corpus import wordnet
from nltk.tokenize import wordpunct_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

#nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/teodorastereciu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [32]:
# helper function to convert the pos tag format into something compatible with the lemmatizer
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# used to clean the text of a conversation
def clean_text(doc, lemmatizer):
    # expand contractions
    try:
        doc = contractions.fix(doc)
    except:
        if DEBUG:
            print(doc)
        return
    
    # remove every char that is not alphanumeric, keep spaces
    doc = re.sub(r'\W+', ' ', doc)
    
    # tokenize into all lowercase tokens
    tokens = wordpunct_tokenize(doc) 
    tokens = [token.lower() for token in tokens]
    clean_tokens = ['<' + token + '>' if token in ['sol', 'eol'] else token for token in tokens]
    
    # lemmatize tokens based on pos tags
    # aka get base form 
    '''pos = pos_tag(tokens)
    clean_tokens = []
    for item in pos:
        word = item[0]
        tag = item[1]
        if word in ['sol', 'eol']:
            clean_tokens.append('<' + word + '>')
        else:
            clean_tokens.append(lemmatizer.lemmatize(word, get_wordnet_pos(tag)))
    #clean_tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in pos]
    '''
    return clean_tokens

wn_lemmatizer = WordNetLemmatizer()
df['clean_conversation'] = df['conversation'].apply(lambda convo: clean_text(convo, wn_lemmatizer))

In [33]:
df.head()

,conversation,emotions,clean_conversation
0,"<sol> Alright , so I am back in high school ,...","[neutral, neutral, surprise, surprise, anger, ...","[<sol>, alright, so, i, am, back, in, high, sc..."
1,"<sol> I do not want to be single , okay ? I j...","[sadness, neutral, surprise]","[<sol>, i, do, not, want, to, be, single, okay..."
2,"<sol> Oh my God ! <eol> <sol> I know , I kno...","[surprise, neutral, sadness, neutral, neutral,...","[<sol>, oh, my, god, <eol>, <sol>, i, know, i,..."
3,"<sol> Barry , I am sorry ... <eol> <sol> I a...","[sadness, sadness, sadness]","[<sol>, barry, i, am, sorry, <eol>, <sol>, i, ..."
4,"<sol> Oh , look , wish me luck ! <eol> <sol>...","[joy, neutral, joy]","[<sol>, oh, look, wish, me, luck, <eol>, <sol>..."
